<a href="https://colab.research.google.com/github/luzmariacabral/Amazon_Vine_Analysis/blob/main/Vine_Review_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 2.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.0'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:12 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Ign:13 https://developer.download.nvidia.com/compute/c

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-07-05 23:09:51--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  4.72MB/s    in 0.2s    

2021-07-05 23:09:51 (4.72 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Digital_Ebook_Purchase_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|    product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   35471030|R1C5OK9AND7PRP|B00AHK07V0|     175130663|Hunter's Moon (A ...|Digital_Ebook_Pur...|          5|            0|          0|   N|                Y|Shugak is like a ...|This is the most ...| 2015-08-31|
|         US|   26579324|R333RNBQMPVUFT|B014085WTQ|     859232728|Flying Toward For...|Digital_Ebook_Pur

In [5]:
# Create the vine_table. DataFrame
vine_df = df.select(['review_id', 'star_rating', 'helpful_votes', 'total_votes', 'vine', 'verified_purchase'])
vine_df.show(10)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1C5OK9AND7PRP|          5|            0|          0|   N|                Y|
|R333RNBQMPVUFT|          5|            0|          0|   N|                Y|
|R2A2K0GTNSKXM1|          5|            0|          0|   N|                N|
|R365LAQ9REV876|          3|            0|          0|   N|                Y|
|R294OF3SNH6SWZ|          4|            0|          0|   N|                Y|
|R2OMRH58WYEEP9|          5|            0|          0|   N|                N|
| RBFXYUJZIH5JR|          5|            0|          0|   N|                Y|
|R2K2RSBNV42HU4|          5|            0|          0|   N|                N|
|R2CLRFFJ5HJSU3|          4|            0|          0|   N|                N|
| RS1O7V45AADDO|          4|            0|          0|   N|     

In [6]:
#Filtering vine_df, total_votes greater than or equal to 20
vineVotes_df=vine_df.filter("total_votes>=20")

In [7]:
#Filter totalVotes_vine where all helpful_votes/total_votes are >+50%
#highVotes_df=vineVotes_df.withcolumn("%_of_votes", (vineVotes_df["helpful_votes"]/vineVotes_df["total_votes"])*100).show()
#test_df=highVotes_df.loc[highVotes_df["%_of_votes"] >= 50]
#test_df = vineVotes_df.filter[(vineVotes_df["helpful_votes"]/vineVotes_df["total_votes"] >= 0.5)]
highVotes_df = vineVotes_df.filter(vineVotes_df["helpful_votes"]/vineVotes_df["total_votes"] >= 0.5)


In [8]:
paid_vine_df=highVotes_df.filter(highVotes_df["vine"]=="Y").show()

+---------+-----------+-------------+-----------+----+-----------------+
|review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+---------+-----------+-------------+-----------+----+-----------------+
+---------+-----------+-------------+-----------+----+-----------------+



In [9]:
non_vine_df = highVotes_df.filter(highVotes_df["vine"] == "N").show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2HQLKREFKG25D|          5|           21|         21|   N|                Y|
| RWIL1XU6YM0U4|          5|           20|         20|   N|                N|
|R28SB7Q1WBJRFM|          5|           25|         27|   N|                N|
|R3PVKAWMZW55U8|          5|          106|        119|   N|                Y|
|R21JS1HIQV1H7W|          3|           17|         22|   N|                Y|
|R245IE9DATP3ZL|          3|           15|         23|   N|                Y|
| RB1Y3RO4J4854|          3|           23|         30|   N|                Y|
|R23P75H9VX5RLM|          5|           32|         36|   N|                Y|
|R3JCH5P63TIQ4E|          5|           19|         20|   N|                N|
| RYA7IZ8VHN5P8|          2|           37|         42|   N|     

In [11]:
#Determine the total number of reviews
total_reviews = vine_df.count()
total_reviews


12520722

In [35]:
#Determine number of 5-star reviews
total_fivestar_df = vine_df.filter(vine_df["star_rating"]==5)
# total_fivestar_df.show()
fivestar_count = total_fivestar_df.count()
fivestar_count

7679679

In [41]:
#Determining number of PAID 5-star review
total_paid_5star = total_fivestar_df.filter(total_fivestar_df['vine'] == 'Y')
fivestar_paid = total_paid_5star.count()

#Dividing number of PAID 5-star review by total # of 5 star reviews to get percentage of paid 5-star review
percent_fivestar_pd = fivestar_paid / fivestar_count
percent_fivestar_pd

2.2136341896581875e-06

In [42]:
#Determining number of UNPAID 5-star reviews
total_unpaid_fivestar = total_fivestar_df.filter(total_fivestar_df['vine']=='N')
fivestar_unpaid = total_unpaid_fivestar.count()

#Dividing # of UNPAID by total # of 5star reviews to get percentage of UNPAID reviews
percent_fivestar_unpaid = fivestar_unpaid / fivestar_count
percent_fivestar_unpaid

0.9999977863658104